# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/missile/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/missile/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [25]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [26]:
# load data from database
table_name = "Message"
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table(table_name, engine)
df.shape

(26216, 41)

In [35]:
# Try to get a distribution of each category

td = df.copy()
td.drop(["index", "id", "message", "original", "genre"], axis=1, inplace=True)
sum_row = {col: td[col].sum() for col in td}
pct_row = {col: (td[col].sum()/df.shape[0])*100 for col in td}
# Turn the sums into a DataFrame with one row with an index of 'Total':
cat_dist = pd.DataFrame([sum_row, pct_row], index=["Total", "Pct"])
cat_dist.head()
#sum_df = pd.DataFrame()
#for col in td:
#    sum_df[col] = td[col].sum()
#sum_df.head()    

,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,...,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
Total,309.00000,10860.000000,1333.000000,0.0,405.000000,530.000000,1194.000000,5075.000000,2455.00000,532.000000,...,4474.000000,724.000000,471.000000,2314.000000,120.000000,2443.000000,159.0000,1201.000000,1672.000000,7297.000000
Pct,1.17867,41.425084,5.084681,0.0,1.544858,2.021666,4.554471,19.358407,9.36451,2.029295,...,17.065914,2.761672,1.796613,8.826671,0.457736,9.318737,0.6065,4.581172,6.377785,27.834147


In [51]:
import plotly
import plotly.graph_objs as go

plotly.offline.plot({
    "data": [go.Bar(x=cat_dist.columns.values, y=cat_dist.iloc[0])]
}, auto_open=True)

'file:///Users/missile/Udacity/udacity_dsnd/disaster_proj/temp-plot.html'

In [52]:
# Try to get a distribution of number of categories per message
td["categories"].values[:] = 0
td["categories"] = td.sum(axis=1)
vc = td["categories"].value_counts()
vc

0     6122
1     5121
3     3609
5     2956
4     2276
6     1903
7     1347
8      880
2      631
9      577
10     344
11     183
12     107
13      57
14      38
15      20
17      12
16       8
18       7
19       5
21       4
20       4
27       2
22       2
24       1
Name: categories, dtype: int64

In [58]:
#vc.values
#vc.index.values
plotly.offline.plot({
    "data": [go.Bar(x=vc.index.values, y=vc.values)]
}, auto_open=True)

'file:///Users/missile/Udacity/udacity_dsnd/disaster_proj/temp-plot.html'

In [27]:
df[df.isnull().any(axis=1)]

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7423,7483,8365,NOTES: It mark as not enough information,None,direct,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9891,9956,11186,My thoughts and prayers go out to all the live...,None,social,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9892,9957,11188,I m sorry for the poor people in Haiti tonight...,None,social,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9893,9958,11189,RT selenagomez UNICEF has just announced an em...,None,social,1,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
9894,9959,11192,lilithia yes 5.2 magnitude earthquake hit mani...,None,social,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9895,9960,11193,RT TheNewsBlotter RT caribnews On Call Interna...,None,social,1,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
9896,9961,11195,Most Eureka homeowners won&#39 t collect on qu...,None,social,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9897,9962,11196,Haiti hit by largest earthquake in over 200 ye...,None,social,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9898,9963,11197,RT pinkelephantpun Earthquake Relief Donate ht...,None,social,1,1,0,1,0,...,0,0,1,0,0,0,1,0,0,0
9899,9964,11198,Praying that today s earthquake in #Haiti mini...,None,social,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [51]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# One Hot Encode genre

df = pd.concat([df,pd.get_dummies(df["genre"], prefix="genre")], axis=1)
df.head()

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,floods,storm,fire,earthquake,cold,other_weather,direct_report,genre_direct,genre_news,genre_social
0,0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,...,0,1,0,0,0,0,0,1,0,0
2,2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,...,0,0,0,0,0,0,0,1,0,0
4,4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [4]:
# Feature
X = df.message.values  # And "original" and "genre" -- ?
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [5]:
# Target
df.drop(["index", "id", "message", "original", "genre"], axis=1, inplace=True)
y = df.values
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [6]:
category_list = df.columns.values
category_list

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'],
      dtype=object)

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [56]:
#forest = RandomForestClassifier(n_estimators=10, random_state=1)
#pipeline = Pipeline([
#    ('vect', CountVectorizer(tokenizer=tokenize)),
#    ('tfidf', TfidfTransformer()),
#    ('clf', MultiOutputClassifier(forest))
#])

In [8]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])



### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [10]:
pipeline.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The de

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
y_pred = pipeline.predict(X_test)
print(y_pred.__class__)
y_pred

<class 'numpy.ndarray'>


array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [12]:
category_cnt = len(y_pred[0])
print(category_cnt)
category_nums = np.arange(0,category_cnt)
print(category_nums)

36
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35]


In [13]:
y_pred_df = pd.DataFrame(y_pred, columns=category_list) #np.arange(0,len(y_pred[0])))
              #index=np.arange(0,y_pred.size),    # 0-size of array
              #columns=np.arange(0,len(y_pred[0])))  # 0-num of cols
y_pred_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y_test_df = pd.DataFrame(y_test, columns=category_list) #np.arange(0,len(y_test[0])))
              #index=np.arange(0,y_pred.size),    # 0-size of array
              #columns=np.arange(0,len(y_pred[0])))  # 0-num of cols
y_test_df.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
1,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
y_test_df.shape

(6554, 36)

In [16]:
cols = y_pred_df.columns.values
cols

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'],
      dtype=object)

In [17]:
for col in y_pred_df.columns.values:
    print("Category: ", col)
    col_pred = y_pred_df[col].tolist()
    col_test = y_test_df[col].tolist()
    print(classification_report(col_test, col_pred))

Category:  related
              precision    recall  f1-score   support

           0       0.62      0.35      0.44      1547
           1       0.82      0.93      0.87      4967
           2       0.73      0.20      0.31        40

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.72      0.49      0.54      6554
weighted avg       0.77      0.79      0.77      6554

Category:  request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5438
           1       0.82      0.39      0.53      1116

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.85      0.69      0.73      6554
weighted avg       0.88      0.88      0.86      6554

Category:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6530
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      6554
   macro avg       

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        43

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

Category:  hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6494
           1       0.00      0.00      0.00        60

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554

Category:  shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6509
           1       0.00      0.00      0.00        45

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99   

In [30]:
from yellowbrick.classifier import ClassificationReport
visualizer = ClassificationReport(pipeline, classes=cols, support=True)

visualizer.fit(X_train, y_train)  # Fit the visualizer and the model
visualizer.score(X_test, y_test)  # Evaluate the model on the test data
g = visualizer.poof()             # Draw/show/poof the data

/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The de

ValueError: multiclass-multioutput is not supported

### 6. Improve your model
Use grid search to find better parameters. 

In [18]:
# Other params:
#'vect__ngram_range': ((1, 1), (1, 2)),
#'vect__max_df': (0.5, 0.75, 1.0),
#'vect__max_features': (None, 5000, 10000),
#'tfidf__use_idf': (True, False),
# 'clf__estimator__min_samples_split': [2, 3, 4],

parameters = {
    'clf__estimator__n_estimators': [100, 200],
    #'clf__estimator__min_samples_split': [2, 3],
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=1, cv=5, verbose=100)

In [19]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=100, score=0.24078311721332316, total= 4.0min
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.0min remaining:    0.0s
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=100, score=0.24561403508771928, total= 4.0min
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 10.0min remaining:    0.0s
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=100, score=0.24084435401831128, total= 4.0min
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 14.9min remaining:    0.0s
[CV] clf__estimator__n_estimators=100 ................................
[CV]  clf__estimator__n_estimators=100, score=0.24186164801627671, total= 4.1min
[Parallel(n

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'clf__estimator__n_estimators': [100, 200]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=100)

In [20]:
print(grid_search.best_params_)

{'clf__estimator__n_estimators': 200}


In [27]:
sorted(pipeline.get_params().keys())

['clf',
 'clf__estimator',
 'clf__estimator__bootstrap',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'clf__estimator__oob_score',
 'clf__estimator__random_state',
 'clf__estimator__verbose',
 'clf__estimator__warm_start',
 'clf__n_jobs',
 'memory',
 'steps',
 'tfidf',
 'tfidf__norm',
 'tfidf__smooth_idf',
 'tfidf__sublinear_tf',
 'tfidf__use_idf',
 'vect',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__s

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
y_pred = grid_search.predict(X_test)

In [22]:
for col in y_pred_df.columns.values:
    print("Category: ", col)
    col_pred = y_pred_df[col].tolist()
    col_test = y_test_df[col].tolist()
    print(classification_report(col_test, col_pred))

Category:  related
              precision    recall  f1-score   support

           0       0.62      0.35      0.44      1547
           1       0.82      0.93      0.87      4967
           2       0.73      0.20      0.31        40

   micro avg       0.79      0.79      0.79      6554
   macro avg       0.72      0.49      0.54      6554
weighted avg       0.77      0.79      0.77      6554

Category:  request
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      5438
           1       0.82      0.39      0.53      1116

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.85      0.69      0.73      6554
weighted avg       0.88      0.88      0.86      6554

Category:  offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6530
           1       0.00      0.00      0.00        24

   micro avg       1.00      1.00      1.00      6554
   macro avg       

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6511
           1       0.00      0.00      0.00        43

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

Category:  hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6494
           1       0.00      0.00      0.00        60

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554

Category:  shops
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6509
           1       0.00      0.00      0.00        45

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99   

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# TODO: add genre categorical feature
# Look at using XGBoost? or build Neural Net?

### 9. Export your model as a pickle file

In [24]:
import pickle
filename = 'model.pkl'
pickle.dump(grid_search, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.